#### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import comb

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.model_selection import(
    KFold,
    RepeatedKFold,
    LeaveOneOut,
    LeavePOut,
    StratifiedKFold,
    cross_validate,
    train_test_split,
)

#### Load the dataset

In [ ]:
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
y.value_counts() / len(y)

0    0.627417
1    0.372583
Name: count, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

#### K-Fold Cross-Validation

In [ ]:
logit = LogisticRegression(penalty = 'l2', C=10, solver='liblinear', random_state=42, max_iter=10000)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

clf = cross_validate(
    logit,
    X_train,
    y_train,
    scoring='accuracy',
    return_train_score=True,
    cv=kf,
)

clf['test_score']

array([1.        , 0.95      , 0.9375    , 0.92405063, 0.97468354])

In [ ]:
clf['train_score']

array([0.95597484, 0.97169811, 0.96855346, 0.97492163, 0.95611285])

In [ ]:
print('mean train set accuracy:', np.mean(clf['train_score']), ' +- ', np.std(clf['train_score']))
print('mean test set accuracy:', np.mean(clf['test_score']), ' +- ', np.std(clf['test_score']))

mean train set accuracy: 0.9654521795705922  +-  0.007941574843749095
mean test set accuracy: 0.9572468354430381  +-  0.027115729354028017


#### Repeated K-Fold

In [ ]:
logit = LogisticRegression(penalty='l2', C=1, solver='liblinear', random_state=42, max_iter=10000)

rkf = RepeatedKFold(
    n_splits=5,
    n_repeats=10,
    random_state=42,
    )

print('We expect K * n performance metrics: ', 5*10)

clf = cross_validate(
    logit,
    X_train,
    y_train,
    scoring='accuracy',
    return_train_score=True,
    cv=rkf,
)

print('Number of metrics obtained: ', len(clf['test_score']))

clf['test_score']

We expect K * n performance metrics:  50
Number of metrics obtained:  50


array([1.        , 0.95      , 0.9       , 0.92405063, 0.96202532,
       0.925     , 0.975     , 0.975     , 0.94936709, 0.93670886,
       0.975     , 0.8875    , 0.95      , 0.96202532, 0.97468354,
       0.95      , 0.95      , 0.975     , 0.91139241, 0.97468354,
       0.975     , 0.9625    , 0.9125    , 0.93670886, 0.91139241,
       0.9125    , 0.95      , 0.9375    , 0.94936709, 0.94936709,
       0.9875    , 0.9875    , 0.9375    , 0.89873418, 0.94936709,
       0.95      , 0.975     , 0.925     , 0.92405063, 0.93670886,
       0.9625    , 0.95      , 0.95      , 0.94936709, 0.91139241,
       0.9       , 0.95      , 0.9875    , 0.96202532, 0.93670886])

In [ ]:
print('mean train set accuracy:', np.mean(clf['train_score']), ' +- ', np.std(clf['train_score']))
print('mean test set accuracy:', np.mean(clf['test_score']), ' +- ', np.std(clf['test_score']))

mean train set accuracy: 0.9604877664083911  +-  0.006896279768373815
mean test set accuracy: 0.9467025316455697  +-  0.026201678577197657


#### Leave One Out

In [ ]:
logit = LogisticRegression(penalty='l2', C=1, solver='liblinear', random_state=42, max_iter=10000)
loo = LeaveOneOut()

print('We expect as many metrics as data in the train set: ', len(X_train))

clf = cross_validate(
    logit,
    X_train,
    y_train,
    scoring='accuracy',
    return_train_score=True,
    cv=loo,
)

print('Number of metrics obtained: ', len(clf['test_score']))
len(clf['test_score'])

We expect as many metrics as data in the train set:  398
Number of metrics obtained:  398


398

In [ ]:
print('mean train set accuracy:', np.mean(clf['train_score']), ' +- ', np.std(clf['train_score']))
print('mean test set accuracy:', np.mean(clf['test_score']), ' +- ', np.std(clf['test_score']))

mean train set accuracy: 0.9575332582307002  +-  0.0012506853463975798
mean test set accuracy: 0.9447236180904522  +-  0.22851893469149895


#### Leave P Out

In [ ]:
logit = LogisticRegression(penalty='l2', C=1, solver='liblinear', random_state=42, max_iter=10000)
lpo = LeavePOut(p=2)

print('We expect :', comb(100, 2), 'metrics')

X_train_small = X_train.head(100)
y_train_small = y_train.head(100)

clf = cross_validate(
    logit,
    X_train_small,
    y_train_small,
    scoring='accuracy',
    return_train_score=True,
    cv=lpo,
)

print('Number of metrics obtained: ', len(clf['test_score']))

We expect : 4950.0 metrics
Number of metrics obtained:  4950


In [ ]:
print('mean train set accuracy:', np.mean(clf['train_score']), ' +- ', np.std(clf['train_score']))
print('mean test set accuracy:', np.mean(clf['test_score']), ' +- ', np.std(clf['test_score']))

mean train set accuracy: 0.9700041228612658  +-  0.0032331283783864494
mean test set accuracy: 0.918989898989899  +-  0.19306928127674033


#### Stratified K-Fold Cross-Validation

In [ ]:
logit = LogisticRegression(penalty='l2', C=1, solver='liblinear', random_state=42, max_iter=10000)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

clf = cross_validate(
    logit,
    X_train,
    y_train,
    scoring='accuracy',
    return_train_score=True,
    cv=skf,
)

len(clf['test_score'])

In [ ]:
print('mean train set accuracy:', np.mean(clf['train_score']), ' +- ', np.std(clf['train_score']))
print('mean test set accuracy:', np.mean(clf['test_score']), ' +- ', np.std(clf['test_score']))